In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# download the dataset to your folder or use it on kaggle notebook directly

train_file = np.load('../../../train.npz')
# train_file = np.load('/kaggle/input/cse-251-b-2025/train.npz')
train_data = train_file['data']

test_file = np.load('../../../test_input.npz')
# test_file = np.load('/kaggle/input/cse-251-b-2025/test_input.npz')
test_data = test_file['data']

In [4]:
print(train_data.shape, test_data.shape)

# Split once for later use
X_train = train_data[..., :50, :5] # ignore the 6th element when convolving
Y_train = train_data[:, 0, 50:, :2]

(10000, 50, 110, 6) (2100, 50, 50, 6)


In [5]:
import sys
sys.path.append('../..')
from TrajectoryDataset import TrajectoryDatasetTrain, TrajectoryDatasetTest
from utils import train_model

sys.path.append('../../..')
from models.simple_cnn import SimpleCNNModel

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch_geometric.data import Batch

In [7]:
torch.manual_seed(251)
np.random.seed(42)

scale = 5.0

N = len(train_data)
val_size = int(0.05 * N)
train_size = N - val_size

train_dataset = TrajectoryDatasetTrain(train_data[:train_size], scale=scale, augment=False, center_agent=0, scene=0)
val_dataset = TrajectoryDatasetTrain(train_data[train_size:], scale=scale, augment=False, center_agent=0, scene=0)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=lambda x: Batch.from_data_list(x))
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=lambda x: Batch.from_data_list(x))

# Set device for training speedup
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using Apple Silicon GPU")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using CUDA GPU")
else:
    device = torch.device('cpu')

Using Apple Silicon GPU


In [8]:
simple_cnn_model = SimpleCNNModel(train_data.shape[3])

# Define loss function and optimizer
simple_cnn_criterion = nn.MSELoss()  # For regression task

simple_cnn_optimizer = optim.AdamW(simple_cnn_model.parameters(), lr=0.0001, weight_decay=1e-4)

simple_cnn_scheduler = CosineAnnealingLR(simple_cnn_optimizer, T_max=100, eta_min=1e-6)

early_stopping_patience = 20

In [9]:
simple_cnn_model = simple_cnn_model.to(device)
train_model(
    simple_cnn_model, 
    train_dataloader, 
    val_dataloader, 
    device, 
    simple_cnn_optimizer, 
    simple_cnn_criterion, 
    simple_cnn_scheduler, 
    early_stopping_patience,
    "simple_cnn_weights"
)

Epoch:   1%|          | 1/100 [00:06<10:19,  6.26s/epoch]

Epoch 000 | Learning rate 0.000100 | train normalized MSE  63.4818 | val normalized MSE  64.1759, | val unnormalized MAE  25.9164 | val unnormalized MSE 1604.3965


Epoch:   2%|▏         | 2/100 [00:08<06:44,  4.12s/epoch]

Epoch 001 | Learning rate 0.000100 | train normalized MSE  62.4363 | val normalized MSE  64.1716, | val unnormalized MAE  25.9113 | val unnormalized MSE 1604.2910


Epoch:   3%|▎         | 3/100 [00:11<05:47,  3.58s/epoch]

...

Epoch 078 | Learning rate 0.000011 | train normalized MSE   3.2020 | val normalized MSE   1.0759, | val unnormalized MAE   3.3745 | val unnormalized MSE  26.8985


Epoch:  80%|████████  | 80/100 [04:08<01:09,  3.48s/epoch]

Epoch 079 | Learning rate 0.000010 | train normalized MSE   3.1111 | val normalized MSE   1.0521, | val unnormalized MAE   3.3302 | val unnormalized MSE  26.3035


Epoch:  80%|████████  | 80/100 [04:11<01:02,  3.15s/epoch]

Epoch 080 | Learning rate 0.000010 | train normalized MSE   3.2185 | val normalized MSE   1.0319, | val unnormalized MAE   3.2744 | val unnormalized MSE  25.7981
Early stop!


In [10]:
test_dataset = TrajectoryDatasetTest(test_data, scale=scale, center_agent=0, scene=0)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False,
                         collate_fn=lambda xs: Batch.from_data_list(xs))

simple_cnn_weights = torch.load("simple_cnn_weights.pt")
simple_cnn_model.load_state_dict(simple_cnn_weights)

# also save entire model
torch.save(simple_cnn_model, "full_simple_cnn_model.pt")

test_dataset.generate_submission_predictions(simple_cnn_model, device, output_file_name='simple_cnn_submission')